In [53]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

##### Make a dataframe with date range between 1-1-2001 to 31-8-2022

In [54]:
begin = datetime(2001,1,1)
end = datetime(2022,8,31) 

In [55]:
df = pd.DataFrame({'date':pd.date_range(begin, end)})
df.set_index('date', inplace = True)

In [56]:
def refill_dates(df, index_name):
    df = df.dropna()
    df.set_axis(['date', index_name], axis=1, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace = True)    
    idx = pd.date_range(begin, end)
    df = df.reindex(idx, fill_value=None)
    assert len(df.dropna(subset=[index_name])) > 0
    return df


In [57]:
def merge_to_master(master_df, ind_df, index_name):
    print(index_name)
    ind_df = refill_dates(ind_df, index_name)
    master_df = master_df.merge(ind_df, how='outer', left_index=True, right_index=True)
    return master_df   
    

##### Compile Inflation

In [58]:
inflation = pd.read_excel('./inflation.xlsx')
inflation

,START&END DATE,Unnamed: 1,CPURNSA Index,Unnamed: 3,RR2YCUS Index,Unnamed: 5,EHGDUSY Index,Unnamed: 7
0,2001-12-30 00:00:00,NaN,2001-12-31,176.7,2001-12-31,0.29,2001-12-31,1.0
1,2022-08-25 00:00:00,NaN,2002-01-31,177.1,2002-01-01,0.49,2002-12-31,1.7
2,NaN,NaN,2002-02-28,177.8,2002-01-02,0.64,2003-12-31,2.8
3,LEGEND,NaN,2002-03-31,178.8,2002-01-03,0.61,2004-12-31,3.9
4,CPURNSA Index,CPI,2002-04-30,179.8,2002-01-04,0.59,2005-12-31,3.5
...,...,...,...,...,...,...,...,...
5383,NaN,NaN,NaT,NaN,2022-08-19,-3.09,NaT,NaN
5384,NaN,NaN,NaT,NaN,2022-08-22,-3.01,NaT,NaN
5385,NaN,NaN,NaT,NaN,2022-08-23,-3.02,NaT,NaN
5386,NaN,NaN,NaT,NaN,2022-08-24,-2.93,NaT,NaN


In [59]:
cpurnsa = inflation[['CPURNSA Index', 'Unnamed: 3']]
rr2ycus = inflation[['RR2YCUS Index', 'Unnamed: 5']]
ehgdusy = inflation[['EHGDUSY Index', 'Unnamed: 7']]
df = merge_to_master(df, cpurnsa, 'CPURNSA Index')
df = merge_to_master(df, rr2ycus, 'RR2YCUS Index')
df = merge_to_master(df, ehgdusy, 'EHGDUSY Index')

CPURNSA Index
RR2YCUS Index
EHGDUSY Index


In [60]:
df

,CPURNSA Index,RR2YCUS Index,EHGDUSY Index
date,,,
2001-01-01,NaN,NaN,NaN
2001-01-02,NaN,NaN,NaN
2001-01-03,NaN,NaN,NaN
2001-01-04,NaN,NaN,NaN
2001-01-05,NaN,NaN,NaN
...,...,...,...
2022-08-27,NaN,NaN,NaN
2022-08-28,NaN,NaN,NaN
2022-08-29,NaN,NaN,NaN


##### Compile INDPRO

In [61]:
production = pd.read_csv('./INDPRO.csv')
production

,DATE,INDPRO
0,2000-01-01,91.6261
1,2000-02-01,91.9626
2,2000-03-01,92.3118
3,2000-04-01,92.8872
4,2000-05-01,93.1620
...,...,...
267,2022-04-01,104.2577
268,2022-05-01,104.1868
269,2022-06-01,104.1778
270,2022-07-01,104.7189


In [62]:
indpro = production[['DATE', 'INDPRO']]
df = merge_to_master(df, indpro, 'INDPRO')

INDPRO


In [63]:
df.dropna(subset=['INDPRO'])

,CPURNSA Index,RR2YCUS Index,EHGDUSY Index,INDPRO
date,,,,
2001-01-01,NaN,NaN,NaN,92.0304
2001-02-01,NaN,NaN,NaN,91.4079
2001-03-01,NaN,NaN,NaN,91.1751
2001-04-01,NaN,NaN,NaN,90.8767
2001-05-01,NaN,NaN,NaN,90.3750
...,...,...,...,...
2022-04-01,NaN,-3.70,NaN,104.2577
2022-05-01,NaN,NaN,NaN,104.1868
2022-06-01,NaN,-3.28,NaN,104.1778


##### Compile Money Supply

In [64]:
m1s = pd.read_csv("./M1SL.csv")
m1s

,DATE,M1SL
0,1959-01-01,138.9
1,1959-02-01,139.4
2,1959-03-01,139.7
3,1959-04-01,139.7
4,1959-05-01,140.7
...,...,...
759,2022-04-01,20615.3
760,2022-05-01,20620.8
761,2022-06-01,20545.0
762,2022-07-01,20516.0


In [65]:
m1 = m1s[['DATE', 'M1SL']]
df = merge_to_master(df, m1, 'M1')

M1


In [66]:
m2s = pd.read_csv("./M2SL.csv")
m2s

,DATE,M2SL
0,1959-01-01,286.6
1,1959-02-01,287.7
2,1959-03-01,289.2
3,1959-04-01,290.1
4,1959-05-01,292.2
...,...,...
759,2022-04-01,21655.6
760,2022-05-01,21684.5
761,2022-06-01,21667.3
762,2022-07-01,21709.6


In [67]:
m2 = m2s[['DATE', 'M2SL']]
df = merge_to_master(df, m2, 'M2')

M2


##### Compile Bill Rates

In [68]:
bill = pd.read_csv("bill-rates-2002-2021.csv")
bill

,Date,4 WEEKS BANK DISCOUNT,4 WEEKS COUPON EQUIVALENT,8 WEEKS BANK DISCOUNT,8 WEEKS COUPON EQUIVALENT,13 WEEKS BANK DISCOUNT,13 WEEKS COUPON EQUIVALENT,26 WEEKS BANK DISCOUNT,26 WEEKS COUPON EQUIVALENT,52 WEEKS BANK DISCOUNT,52 WEEKS COUPON EQUIVALENT
0,12/31/21,0.06,0.06,0.05,0.05,0.06,0.06,0.19,0.19,0.38,0.39
1,12/30/21,0.06,0.06,0.06,0.06,0.05,0.05,0.19,0.19,0.37,0.38
2,12/29/21,0.01,0.01,0.02,0.02,0.05,0.05,0.19,0.19,0.38,0.39
3,12/28/21,0.03,0.03,0.04,0.04,0.06,0.06,0.20,0.20,0.38,0.39
4,12/27/21,0.04,0.04,0.05,0.05,0.06,0.06,0.21,0.21,0.30,0.30
...,...,...,...,...,...,...,...,...,...,...,...
5001,1/8/02,1.67,1.69,NaN,NaN,1.65,1.68,1.74,1.78,NaN,NaN
5002,1/7/02,1.68,1.71,NaN,NaN,1.66,1.69,1.73,1.77,NaN,NaN
5003,1/4/02,1.69,1.72,NaN,NaN,1.69,1.72,1.78,1.82,NaN,NaN
5004,1/3/02,1.70,1.73,NaN,NaN,1.70,1.73,1.78,1.82,NaN,NaN


In [69]:
billRate_13wk = bill[['Date', '13 WEEKS BANK DISCOUNT']]
df = merge_to_master(df, billRate_13wk, 'Bill Rate 13 week')

Bill Rate 13 week


##### Compile Unemployment

In [70]:
unemp = pd.read_csv("unemploymentUS.csv")
unemp

,DATE,UNRATE
0,1/1/1948,3.4
1,2/1/1948,3.8
2,3/1/1948,4.0
3,4/1/1948,3.9
4,5/1/1948,3.5
...,...,...
892,5/1/2022,3.6
893,6/1/2022,3.6
894,7/1/2022,3.5
895,8/1/2022,3.7


In [71]:
unemp = unemp[['DATE', 'UNRATE']]
df = merge_to_master(df, unemp, 'Unemployment Rate')

Unemployment Rate


In [72]:
df

,CPURNSA Index,RR2YCUS Index,EHGDUSY Index,INDPRO,M1,M2,Bill Rate 13 week,Unemployment Rate
date,,,,,,,,
2001-01-01,NaN,NaN,NaN,92.0304,1096.7,4975.7,NaN,4.2
2001-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-08-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Merge Political Risk

In [73]:
risk = pd.read_excel("./implied_erp.xls", sheet_name='Historical Impl Premiums')
risk

,Date updated:,2022-01-05 00:00:00,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Created by:,"Aswath Damodaran, adamodar@stern.nyu.edu",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,What is this data?,Implied Equity Risk Premiums (by year),NaN,NaN,NaN,S&P 500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Home Page:,http://www.damodaran.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Data website:,https://www.stern.nyu.edu/~adamodar/New_Home_P...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NaN,Period,ERP,ERP + Riskfree Rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,1960-2020,0.042127,0.100645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,2001-2020,0.04948,0.080125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,2011-2020,0.0553,0.07681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
risk = risk[6:68][['Date updated:', 'Unnamed: 17']]
risk['Date updated:'] = pd.to_datetime(risk['Date updated:'], format='%Y')
df = merge_to_master(df, risk, 'ERP')

ERP


##### Merge Financial Indices

In [75]:
##### I DON'T KNOW WHY THE FUCK IS IT LOADING THIS FILE HERE!!!!!!!!!!!!!! 
##### @Elisa does this file have some hidden columns lol from Elisa: -> your are using the wrong file here, please use data_master
finance = pd.read_excel('bloomberg_noformulas.xlsx') # 
finance

,START&END DATE,Unnamed: 1,Unnamed: 2,CPURNSA Index,Unnamed: 4,EHGDUSY Index,Unnamed: 6,RR2YCUS Index,RR10CUS Index,Unnamed: 9,...,Unnamed: 12,XAU BGN Curncy,Unnamed: 14,XAG BGN Curncy,Unnamed: 16,VIX INDEX,Unnamed: 18,MSCI US EQUITY,Unnamed: 20,BBDXY INDEX
0,37255,NaN,2001-12-31,176.7,2001-12-31,1.0,2001-12-31,0.29,2.32,NaN,...,2001-12-31,278.95,2001-12-31,4.62,2001-12-31,23.80,2007-11-14,18.00,2004-12-31,1000.00
1,44798,NaN,2002-01-31,177.1,2002-12-31,1.7,2002-01-01,0.49,2.47,NaN,...,2002-01-01,278.95,2002-01-02,4.60,2002-01-02,22.71,2007-11-15,24.97,2005-01-03,1005.04
2,NaN,NaN,2002-02-28,177.8,2003-12-31,2.8,2002-01-02,0.64,2.60,NaN,...,2002-01-02,278.75,2002-01-03,4.65,2002-01-03,21.34,2007-11-16,24.96,2005-01-04,1018.29
3,LEGEND,NaN,2002-03-31,178.8,2004-12-31,3.9,2002-01-03,0.61,2.55,NaN,...,2002-01-03,278.45,2002-01-04,4.70,2002-01-04,20.45,2007-11-19,25.75,2005-01-05,1018.44
4,CPURNSA Index,CPI,2002-04-30,179.8,2005-12-31,3.5,2002-01-04,0.59,2.57,NaN,...,2002-01-04,278.95,2002-01-07,4.71,2002-01-07,21.94,2007-11-20,27.32,2005-01-06,1024.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5383,NaN,NaN,NaT,NaN,NaT,NaN,2022-08-19,-3.09,-3.35,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
5384,NaN,NaN,NaT,NaN,NaT,NaN,2022-08-22,-3.01,-3.31,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
5385,NaN,NaN,NaT,NaN,NaT,NaN,2022-08-23,-3.02,-3.28,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
5386,NaN,NaN,NaT,NaN,NaT,NaN,2022-08-24,-2.93,-3.22,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN


In [76]:
vix = finance[['Unnamed: 16', 'VIX INDEX']]
msci = finance[['Unnamed: 18', 'MSCI US EQUITY']]
bbdxy = finance[['Unnamed: 20', 'BBDXY INDEX']]
df = merge_to_master(df, vix, 'VIX Index')
df = merge_to_master(df, msci, 'MSCI US Equity')
df = merge_to_master(df, bbdxy, 'BBDXY Index')

VIX Index
MSCI US Equity
BBDXY Index


##### Compile Currencies

In [77]:
xaubgn = finance[['Unnamed: 12', 'XAU BGN Curncy']]
xagbgn = finance[['Unnamed: 14', 'XAG BGN Curncy']]
df = merge_to_master(df, xaubgn, 'XAU BGN Curncy')
df = merge_to_master(df, xagbgn, 'XAG BGN Curncy')

XAU BGN Curncy
XAG BGN Curncy


In [78]:
df

,CPURNSA Index,RR2YCUS Index,EHGDUSY Index,INDPRO,M1,M2,Bill Rate 13 week,Unemployment Rate,ERP,VIX Index,MSCI US Equity,BBDXY Index,XAU BGN Curncy,XAG BGN Curncy
date,,,,,,,,,,,,,,
2001-01-01,NaN,NaN,NaN,92.0304,1096.7,4975.7,NaN,4.2,0.716832,NaN,NaN,NaN,NaN,NaN
2001-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Merge GDP data

In [79]:
def gdpData(sheet):
    df = pd.read_excel("./GDP_World_Commoditieswise.xlsx", sheet_name=sheet)
    df = df.rename(columns={'Country Name': 'Year'})
    df = df.set_index('Year')
    return df[3:]
imp_agr = gdpData("Import_Agriculture")
exp_agr = gdpData("Export_Agriculture")
imp_ls = gdpData("Import_Livestock")
exp_ls = gdpData("Export_Livestock")
imp_ind = gdpData("Import_Industrial Metals")
exp_ind = gdpData("Export_Industrial Metals")
imp_eg = gdpData("Import_Energy")
exp_eg = gdpData("Export_Energy")

In [80]:
merged = imp_agr.merge(exp_agr, left_index=True, right_index=True, how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
merged = merged.merge(imp_ls, left_index=True, right_index=True, how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
merged = merged.merge(exp_ls, left_index=True, right_index=True, how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
merged = merged.merge(imp_ind, left_index=True, right_index=True, how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
merged = merged.merge(exp_ind, left_index=True, right_index=True, how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
merged = merged.merge(imp_eg, left_index=True, right_index=True, how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
merged = merged.merge(exp_eg, left_index=True, right_index=True, how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
merged.reset_index(inplace=True)
merged.Year = pd.to_datetime(merged.Year, format='%Y')

In [81]:
merged

,Year,Canada,China,European Union,Mexico,Japan,"Korea, Rep.",Germany,India,Latin America & Caribbean,Colombia,Saudi Arabia,Russian Federation
0,1960-01-01,40461721692.646797,59716467625.314796,NaN,13040000000,44307342950.400002,3958190758.62419,NaN,37029883876.183899,84505840123.943497,4031152976.63904,NaN,NaN
1,1961-01-01,40934952063.9468,50056868957.673203,NaN,14160000000,53508617739.3778,2417558289.36656,NaN,39232435784.035797,89564538264.044693,4540447761.19403,NaN,NaN
2,1962-01-01,42227447631.915901,47209359005.605698,NaN,15200000000,60723018683.733299,2814318516.60968,NaN,42161481858.081902,98133168723.767502,4955543963.36869,NaN,NaN
3,1963-01-01,45029988561.212402,50706799902.510399,NaN,16960000000,69498131797.333298,3988784572.24835,NaN,48421923459.123497,100055914613,4836166666.66667,NaN,NaN
4,1964-01-01,49377522896.703003,59708343488.504303,NaN,20080000000,81749006381.511093,3458939357.73377,NaN,56480289940.989899,110927451702.945999,5973366666.66667,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2017-01-01,1649265644244.090088,12310409370894.199219,14766273067392.099609,1158913035796.370117,4930837369151.419922,1623901496835.790039,3690849152517.649902,2651472946374.910156,5830760733039.75,311883730690.129028,688586094412.680054,1574199387070.899902
58,2018-01-01,1725329192783.02002,13894817549380.300781,15978719985091.5,1222408203104.300049,5037835383110.969727,1724845615629.26001,3977289455388.22998,2702929718960.459961,5701842678618.30957,334198214706.208984,816578674529.140991,1657329646183.620117
59,2019-01-01,1742015045482.310059,14279937500608,15691850452107.300781,1269404276770.699951,5123318151510.620117,1651422932447.77002,3888326788627.439941,2831552222519.990234,5623606841400.30957,323109543324.320984,803616264791.024048,1693113904262.889893
60,2020-01-01,1645423407568.360107,14687673892882,15300141971364.800781,1087117783073.310059,5040107754084.110352,1637895802792.899902,3846413928653.709961,2667687951796.560059,4743154434209.719727,270299982887.01001,703367841222.555054,1488321875489.73999


In [82]:
for col in merged.columns[1:]:
    cnt = merged[['Year', col]]
    df = merge_to_master(df, cnt, col)

Canada
China
European Union
Mexico
Japan
Korea, Rep.
Germany
India
Latin America & Caribbean
Colombia
Saudi Arabia
Russian Federation


In [83]:
df.to_excel("./predictors.xlsx")